## Init

In [1]:
!pip install datasets transformers pandas tqdm matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 22.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [3]:
squad_v2 = True

In [4]:
def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    return model, tokenizer

In [5]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from tqdm.auto import tqdm

def evaluate_model(model, tokenizer, dataset, subset_size=100):
    nlp = pipeline("question-answering", model=model, tokenizer=tokenizer)

    # Determine if 'validation' split is available, otherwise use 'train'
    #split_to_use = 'validation' if 'validation' in dataset else 'train'
    #split_to_use = 'test'
    split_to_use = 'train'

    # Select a subset from the appropriate split
    subset = dataset[split_to_use].select(range(subset_size))
    metric = load_metric("squad_v2")
    results = []

    for example in tqdm(subset, desc=f"Evaluating ({split_to_use} split)", unit="example"):
        outputs = nlp({
            "question": example["question"],
            "context": example["context"]
        })
        formatted_predictions = {
            "prediction_text": outputs['answer'],
            "id": example["id"],
            "no_answer_probability": 0.0
        }
        formatted_references = {
            "answers": example["answers"],
            "id": example["id"]
        }
        metric.add(prediction=formatted_predictions, reference=formatted_references)

    final_scores = metric.compute()
    return final_scores

In [6]:
# Initialize an empty list to hold the datasets
all_datasets = [
    # ("squad_v2" if squad_v2 else "squad", load_dataset("squad_v2" if squad_v2 else "squad")),

    # ("cuad", load_dataset("cuad")),

    # ("covid_qa_deepset", load_dataset("covid_qa_deepset")),

    ("TriviaQA_SQuAD", load_dataset("Kkordik/TriviaQA_SQuAD")),

    # ("books", load_dataset("subjqa", "books")),
    # ("movies", load_dataset("subjqa", "movies")),
    # ("restaurants", load_dataset("subjqa", "restaurants")),
    # ("electronics", load_dataset("subjqa", "electronics")),
    # ("grocery", load_dataset("subjqa", "grocery")),
    # ("tripadvisor", load_dataset("subjqa", "tripadvisor"))
    # Add other datasets as needed
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14229 [00:00<?, ? examples/s]

In [7]:
# Liste des modèles à évaluer
models_checkpoint = [
    "squeezebert/squeezebert-uncased",
    "google-bert/bert-large-uncased",
    "microsoft/deberta-large"
]

## Squad v2

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (validation split):   0%|          | 0/11873 [00:00<?, ?example/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (validation split):   0%|          | 0/11873 [00:00<?, ?example/s]

Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (validation split):   0%|          | 0/11873 [00:00<?, ?example/s]

Metric                                    HasAns_exact  HasAns_f1  \
Dataset  Model                                                      
squad_v2 google-bert/bert-large-uncased       0.404858   7.303644   
         microsoft/deberta-large              0.269906   7.866964   
         squeezebert/squeezebert-uncased      0.185560   6.393064   

Metric                                    HasAns_total  NoAns_exact  NoAns_f1  \
Dataset  Model                                                                  
squad_v2 google-bert/bert-large-uncased         5928.0     1.295206  1.295206   
         microsoft/deberta-large                5928.0     0.454163  0.454163   
         squeezebert/squeezebert-uncased        5928.0     0.992431  0.992431   

Metric                                    NoAns_total  best_exact  \
Dataset  Model                                                      
squad_v2 google-bert/bert-large-uncased        5945.0   50.071591   
         microsoft/deberta-large               5945.0   50.071591   
         squeezebert/squeezebert-uncased       5945.0   50.071591   

Metric                                    best_exact_thresh    best_f1  \
Dataset  Model                                                           
squad_v2 google-bert/bert-large-uncased                 0.0  50.077206   
         microsoft/deberta-large                        0.0  50.071591   
         squeezebert/squeezebert-uncased                0.0  50.073997   

Metric                                    best_f1_thresh     exact        f1  \
Dataset  Model                                                                 
squad_v2 google-bert/bert-large-uncased              0.0  0.850670  4.295123   
         microsoft/deberta-large                     0.0  0.362166  4.155257   
         squeezebert/squeezebert-uncased             0.0  0.589573  3.688881   

Metric                                      total  
Dataset  Model                                     
squad_v2 google-bert/bert-large-uncased   11873.0  
         microsoft/deberta-large          11873.0  
         squeezebert/squeezebert-uncased  11873.0

## Subjqa

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-ec2161226a33>:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_c

...

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/512 [00:00<?, ?example/s]

Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/512 [00:00<?, ?example/s]

Metric                                       HasAns_exact  HasAns_f1  \
Dataset     Model                                                      
books       google-bert/bert-large-uncased       0.000000   8.783961   
            microsoft/deberta-large              0.000000   9.135216   
            squeezebert/squeezebert-uncased      0.000000   7.959382   
electronics google-bert/bert-large-uncased       0.625000  13.389173   
            microsoft/deberta-large              0.000000   8.291876   
            squeezebert/squeezebert-uncased      0.000000  11.607155   
grocery     google-bert/bert-large-uncased       1.119403  13.851732   
            microsoft/deberta-large              0.373134  11.907818   
            squeezebert/squeezebert-uncased      0.000000   8.263615   
movies      google-bert/bert-large-uncased       2.325581  11.169458   
            microsoft/deberta-large              0.000000  10.337100   
            squeezebert/squeezebert-uncased      0.000000   9.452993   
restaurants google-bert/bert-large-uncased       0.000000  11.024831   
            microsoft/deberta-large              0.000000  10.202790   
            squeezebert/squeezebert-uncased      0.000000  10.404360   
tripadvisor google-bert/bert-large-uncased       0.000000   9.889932   
            microsoft/deberta-large              0.000000   7.866860   
            squeezebert/squeezebert-uncased      0.000000   6.967403   

Metric                                       HasAns_total  NoAns_exact  \
Dataset     Model                                                        
books       google-bert/bert-large-uncased          103.0     1.652893   
            microsoft/deberta-large                 103.0     0.000000   
            squeezebert/squeezebert-uncased         103.0     0.000000   
electronics google-bert/bert-large-uncased          160.0     0.000000   
            microsoft/deberta-large                 160.0     0.000000   
            squeezebert/squeezebert-uncased         160.0     1.010101   
grocery     google-bert/bert-large-uncased          268.0     0.619195   
            microsoft/deberta-large                 268.0     0.000000   
            squeezebert/squeezebert-uncased         268.0     4.643963   
movies      google-bert/bert-large-uncased           86.0     1.463415   
            microsoft/deberta-large                  86.0     0.000000   
            squeezebert/squeezebert-uncased          86.0     0.000000   
restaurants google-bert/bert-large-uncased           87.0     0.558659   
            microsoft/deberta-large                  87.0     0.000000   
            squeezebert/squeezebert-uncased          87.0     0.000000   
tripadvisor google-bert/bert-large-uncased          295.0     0.460829   
            microsoft/deberta-large                 295.0     0.000000   
            squeezebert/squeezebert-uncased         295.0    12.442396   

Metric                                        NoAns_f1  NoAns_total  \
Dataset     Model                                                     
books       google-bert/bert-large-uncased    1.652893        242.0   
            microsoft/deberta-large           0.000000        242.0   
            squeezebert/squeezebert-uncased   0.000000        242.0   
electronics google-bert/bert-large-uncased    0.000000        198.0   
            microsoft/deberta-large           0.000000        198.0   
            squeezebert/squeezebert-uncased   1.010101        198.0   
grocery     google-bert/bert-large-uncased    0.619195        323.0   
            microsoft/deberta-large           0.000000        323.0   
            squeezebert/squeezebert-uncased   4.643963        323.0   
movies      google-bert/bert-large-uncased    1.463415        205.0   
            microsoft/deberta-large           0.000000        205.0   
            squeezebert/squeezebert-uncased   0.000000        205.0   
restaurants google-bert/bert-large-uncased    0.558659        179.0   
            m

## cuad

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-b5f1227cb97d>:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_c

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (test split):   0%|          | 0/100 [00:00<?, ?example/s]

Metric                                   HasAns_exact  HasAns_f1  \
Dataset Model                                                      
cuad    google-bert/bert-large-uncased            0.0   1.548707   
        microsoft/deberta-large                   0.0   3.563727   
        squeezebert/squeezebert-uncased           0.0   2.089198   

Metric                                   HasAns_total  NoAns_exact  NoAns_f1  \
Dataset Model                                                                  
cuad    google-bert/bert-large-uncased           28.0     1.388889  1.388889   
        microsoft/deberta-large                  28.0     0.000000  0.000000   
        squeezebert/squeezebert-uncased          28.0     0.000000  0.000000   

Metric                                   NoAns_total  best_exact  \
Dataset Model                                                      
cuad    google-bert/bert-large-uncased          72.0        72.0   
        microsoft/deberta-large                 72.0        72.0   
        squeezebert/squeezebert-uncased         72.0        72.0   

Metric                                   best_exact_thresh  best_f1  \
Dataset Model                                                         
cuad    google-bert/bert-large-uncased                 0.0     72.0   
        microsoft/deberta-large                        0.0     72.0   
        squeezebert/squeezebert-uncased                0.0     72.0   

Metric                                   best_f1_thresh  exact        f1  \
Dataset Model                                                              
cuad    google-bert/bert-large-uncased              0.0    1.0  1.433638   
        microsoft/deberta-large                     0.0    0.0  0.997844   
        squeezebert/squeezebert-uncased             0.0    0.0  0.584975   

Metric                                   total  
Dataset Model                                   
cuad    google-bert/bert-large-uncased   100.0  
        microsoft/deberta-large          100.0  
        squeezebert/squeezebert-uncased  100.0

## covid_qa_deepset

In [ ]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-fb003a4d1849>:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_c

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (train split):   0%|          | 0/100 [00:00<?, ?example/s]

Metric                                   HasAns_exact  HasAns_f1  \
Dataset Model                                                      
cuad    google-bert/bert-large-uncased            0.0   7.301860   
        microsoft/deberta-large                   0.0   1.685389   
        squeezebert/squeezebert-uncased           0.0   9.398043   

Metric                                   HasAns_total  NoAns_exact  NoAns_f1  \
Dataset Model                                                                  
cuad    google-bert/bert-large-uncased           60.0          0.0       0.0   
        microsoft/deberta-large                  60.0         10.0      10.0   
        squeezebert/squeezebert-uncased          60.0          0.0       0.0   

Metric                                   NoAns_total  best_exact  \
Dataset Model                                                      
cuad    google-bert/bert-large-uncased          40.0        40.0   
        microsoft/deberta-large                 40.0        40.0   
        squeezebert/squeezebert-uncased         40.0        40.0   

Metric                                   best_exact_thresh    best_f1  \
Dataset Model                                                           
cuad    google-bert/bert-large-uncased                 0.0  40.772664   
        microsoft/deberta-large                        0.0  40.116667   
        squeezebert/squeezebert-uncased                0.0  41.167169   

Metric                                   best_f1_thresh  exact        f1  \
Dataset Model                                                              
cuad    google-bert/bert-large-uncased              0.0    0.0  4.381116   
        microsoft/deberta-large                     0.0    4.0  5.011233   
        squeezebert/squeezebert-uncased             0.0    0.0  5.638826   

Metric                                   total  
Dataset Model                                   
cuad    google-bert/bert-large-uncased   100.0  
        microsoft/deberta-large          100.0  
        squeezebert/squeezebert-uncased  100.0

## Kkordik/TriviaQA_SQuAD

In [8]:
# DataFrame to store results
results_df = pd.DataFrame()

# Evaluate each model on each dataset
for dataset_name, dataset in all_datasets:
    for model_name in models_checkpoint:
        model, tokenizer = load_model_and_tokenizer(model_name)
        scores = evaluate_model(model, tokenizer, dataset)
        new_row = pd.DataFrame({
            'Dataset': [dataset_name] * len(scores),
            'Model': [model_name] * len(scores),
            'Metric': list(scores.keys()),
            'Value': list(scores.values())
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Pivot for comparison
results_pivot = results_df.pivot_table(index=['Dataset', 'Model'], columns='Metric', values='Value')
display(results_pivot)  # Using display for better formatting

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-fb003a4d1849>:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_c

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of DebertaForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Evaluating (train split):   0%|          | 0/100 [00:00<?, ?example/s]

Metric                                          HasAns_exact  HasAns_f1  \
Dataset        Model                                                      
TriviaQA_SQuAD google-bert/bert-large-uncased            0.0   0.588235   
               microsoft/deberta-large                   0.0   0.261438   
               squeezebert/squeezebert-uncased           0.0   0.392157   

Metric                                          HasAns_total  NoAns_exact  \
Dataset        Model                                                        
TriviaQA_SQuAD google-bert/bert-large-uncased           85.0          0.0   
               microsoft/deberta-large                  85.0          0.0   
               squeezebert/squeezebert-uncased          85.0          0.0   

Metric                                          NoAns_f1  NoAns_total  \
Dataset        Model                                                    
TriviaQA_SQuAD google-bert/bert-large-uncased        0.0         15.0   
               microsoft/deberta-large               0.0         15.0   
               squeezebert/squeezebert-uncased       0.0         15.0   

Metric                                          best_exact  best_exact_thresh  \
Dataset        Model                                                            
TriviaQA_SQuAD google-bert/bert-large-uncased         15.0                0.0   
               microsoft/deberta-large                15.0                0.0   
               squeezebert/squeezebert-uncased        15.0                0.0   

Metric                                          best_f1  best_f1_thresh  \
Dataset        Model                                                      
TriviaQA_SQuAD google-bert/bert-large-uncased      15.0             0.0   
               microsoft/deberta-large             15.0             0.0   
               squeezebert/squeezebert-uncased     15.0             0.0   

Metric                                          exact        f1  total  
Dataset        Model                                                    
TriviaQA_SQuAD google-bert/bert-large-uncased     0.0  0.500000  100.0  
               microsoft/deberta-large            0.0  0.222222  100.0  
               squeezebert/squeezebert-uncased    0.0  0.333333  100.0